# Green and Red Candle Project
## Coded by Victor Zheng & Supervised under Mohammed Hasan

## Progress
#### 04-17-2021: Debugged last index dateTimeMaxHTP for why it was (0,0,0)
#### 04-18-2021: Worked on getting dateTimeMinLTP - the minimum LTP for a paritcular row in dateTimeMaxHTP
#### 04-19-2021-A: Debugged maxHTPTime == "9:37:00 AM": case - it doesn't select the minLTP after maxtime, it selects the LTP at 9:37AM
#### 04-19-2021-B: For each date, found the dropPerc and map it to number of occurrence and found highest occurrence to find buyPriceGeneral. Then found the profitPerc and mapped it to number of occurences and found highest occurrence to know what protit % we will accept to limit sell at
#### 04-20-2021: Added margins for % change from OTP to minLTP. Merged previous code using datasetToMerged(). Started on profits = [] #(date,profit)
#### 04-21-2021: Worked on profits and netProfit for mostOccurredDropPerc and mostOccuredBuyProfit percents

In [19]:
import pandas as pd
import numpy as np
import math
#df = pd.read_csv('Jan 2020 - Dec 2020 merged SPY - Copy.csv')
#df = pd.read_csv('Jan 2020 - Dec 2020 merged SPY.csv')
df = pd.read_csv('Even-smaller-dataframe.csv')
df

"""
#40% training
#30% validation
#30% cross-validation
trainingPerc = 0.4
validationPerc = 0.3
N = len(df)
trainingSet = df[0:N*0.4]
validationSet = df[N*0.4:N*0.7]
"""

,Date,TimeBarStart,Ticker,CallPut,Strike,ExpirationDate,OpenTradePrice,HighTradePrice,LowTradePrice,CloseTradePrice,UnderOpenBidPrice,UnderOpenAskPrice,UnderCloseBidPrice,UnderCloseAskPrice,VWAP,Volume,TotalTrades
0,20200304,9:30:00 AM,SPY,P,306.5,20200304,2.64,2.76,2.64,2.76,306.05,306.07,305.85,305.87,2.7000,2,2
1,20200304,9:30:00 AM,SPY,P,288.0,20200304,0.04,0.06,0.03,0.03,306.06,306.07,306.19,306.20,0.0418,11,6
2,20200304,9:30:00 AM,SPY,P,279.0,20200304,0.01,0.01,0.01,0.01,305.95,305.97,305.95,305.97,0.0100,25,1
3,20200304,9:30:00 AM,SPY,P,304.5,20200304,1.70,1.70,1.62,1.62,306.15,306.16,306.38,306.40,1.6886,7,2
4,20200304,9:30:00 AM,SPY,P,289.0,20200304,0.03,0.05,0.03,0.05,306.09,306.11,306.17,306.19,0.0389,36,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80348,20200311,4:14:00 PM,SPY,C,274.0,20200311,1.06,1.11,1.02,1.05,274.65,274.71,274.62,274.68,1.0830,23,7
80349,20200311,4:14:00 PM,SPY,C,271.0,20200311,3.75,3.75,3.75,3.75,274.67,274.71,274.67,274.71,3.7500,1,1
80350,20200311,4:14:00 PM,SPY,C,276.0,20200311,0.27,0.28,0.17,0.25,274.70,274.74,274.59,274.60,0.2406,17,8
80351,20200311,4:14:00 PM,SPY,C,282.0,20200311,0.01,0.01,0.01,0.01,274.54,274.59,274.54,274.59,0.0100,3,2


## Get minimum value for call/put between 9:46AM and the timecallmax for that day

In [20]:
#function that compares if time 1 is < time 2
def beforeTime(time1,time2):
    
    if len(time1) != 11:
        time1 = "0" + time1
    if len(time2) != 11:
        time2 = "0" + time2
    
    #check last 2 chars to see if AM or PM
    if time1[-2:] == "AM":
        t1IsAM = True
    else:
        t1IsAM = False
    if time2[-2:] == "AM":
        t2IsAM = True
    else:
        t2IsAM = False
    
    if t1IsAM and not t2IsAM:
        return True
    elif t1IsAM and t2IsAM:
        return True if time1 < time2 else False
    elif not t1IsAM and t2IsAM:
        return False
    else: #t1 is pm and t2 is pm
        return True if time1 < time2 else False

#sample test cases - expected TFFF
print(beforeTime("10:05:00 AM","3:59:00 PM"), beforeTime("10:05:00 PM","3:59:00 PM"), beforeTime("10:05:00 AM","3:59:00 AM"),beforeTime("10:05:00 PM","3:59:00 AM"))

def afterTime(time1,time2):
    if time1 == time2:
        return False
    return not beforeTime(time1,time2)

True False False False


### Starting price difference - green (increase) and red (decrease) indicators for the first 5 mins of each date

In [21]:
dateTimeCandleSP = [] #(date,time,color,StrikePrice,StockPrice)
date = df.iloc[0]["Date"]
openPrice = df.iloc[0]["UnderOpenAskPrice"]
skipDate = False
#indices: 0=Date, 1=TimeBarStart, 13= UnderCloseAskPrice, 11=UnderOpenAskPrice
for index,row in df.iterrows():
    #we are looking for the UnderOpenAskPrice(UOAP) at 9:30:00 AM (the first time it occurs) 
    #we are looking for the UnderOpenAskPrice (UOAP) at 9:36:00 AM (the last time it occurs)
    #then we subtract the 9:35:00 AM time by the 9:30:00 AM time
    
    if date == row[0]:
        if skipDate:
            continue
        #lastPrice = row["UnderCloseAskPrice"]
        prevRow = row
        #if time is past 9:35AM, then use the latest 9:35AM (if it doesn't exist, then use the current row's time)
        time = row[1]
        if not beforeTime(time,"9:36:00 AM"):
            #calculate starting price difference
            
            #if 9:35 exists
            if prevRow[1] == "9:35:00 AM":
                #we take the UnderCloseAskPrice
                lastPrice = prevRow[13]
            else:
                #if 9:35 doesn't exist
                #if 9:36 exists, then take the open price, otherwise skip this date
                if time == "9:36:00 AM":
                    lastPrice = row[11]
                else:
                    skipDate = True
                    continue
            diff = lastPrice - openPrice
            if math.isnan(diff):
                #either openPrice or lastPrice doesn't exist
                skipDate = True
                continue
            #print("date={},lastPrice={}".format(date,lastPrice))
            if diff >= 0:
                #green start
                dateTimeCandleSP.append((date,time,"GREEN",math.ceil(lastPrice),lastPrice))
            else:
                dateTimeCandleSP.append((date,time,"RED",math.floor(lastPrice),lastPrice))
            skipDate = True
    else:
        #we found new date, get the green/red bar for the previous date, reinitialize some variables
        date = row[0]
        openPrice = row[11]
        skipDate = False

In [22]:
#(date,time,color,StrikePrice,StockPrice)
dateTimeCandleSP

[(20200304, '9:36:00 AM', 'RED', 305, 305.36),
 (20200306, '9:36:00 AM', 'GREEN', 294, 293.8),
 (20200311, '9:36:00 AM', 'GREEN', 282, 281.13)]

In [23]:
#if the candle is green then we want to buy a call
#if call then you look for the UnderOpenAskPrice at 9:36:00 AM and round that up to next whole number, this gets the Call_SP
#then we look for the Call_SP and see what the OpenTradePrice is at 9:36:00 AM

#if the candle is red then we want to buy a put
#if put then you look for the UnderOpenAskPrice at 9:36:00 AM and round that down to next whole number, this gets the Put_SP
#then we look for the Put_SP and see what the OpenTradePrice is at 9:36:00 AM

dtcspIdx = 0
dfStartIdx = 0
dateWithOTP = [] #(date, time, CallPut, SP, OTP)
for dtcsp in dateTimeCandleSP:
    #print("dateWithOTP: {}".format(dateWithOTP))
    date, time, color, strikeP = dtcsp[0],dtcsp[1],dtcsp[2], dtcsp[3]
    for index,row in df.iloc[dfStartIdx:].iterrows():
        if color == "GREEN":
            #This is call, get OpenTradePrice(OTP) at 9:36AM
            #Match Date, Time, Call=C, StrikePrice
            if row[0] == date and row[1] == time and row[3] == "C" and row[4] == strikeP:
                dateWithOTP.append((date,time,"C",strikeP,row[6]))
                #update for next iteration of dtcsp
                dfStartIdx = index + 1
                break
        else:
            #This is put, get OpenTradePrice(OTP) at 9:36AM
            #Match Date, Time, Call=P, StrikePrice
            if row[0] == date and row[1] == time and row[3] == "P" and row[4] == strikeP:
                dateWithOTP.append((date,time,"P",strikeP,row[6]))
                #update for next iteration of dtcsp
                dfStartIdx = index + 1
                break

In [24]:
#(date, time, CallPut, SP, OTP)
print("(date, time,CallPut, SP, OTP)")
dateWithOTP

(date, time,CallPut, SP, OTP)


[(20200304, '9:36:00 AM', 'P', 305, 2.26),
 (20200306, '9:36:00 AM', 'C', 294, 2.87),
 (20200311, '9:36:00 AM', 'C', 282, 3.24)]

#### For each date,SP,CallPut in dateWithOTP, find the maximum of the hightradeprice and time (b/w 9:37:00 AM to 3:59:00 PM

In [25]:
dfStartIdx = 0
#Date	TimeBarStart	Ticker	CallPut	Strike	ExpirationDate	OpenTradePrice	HighTradePrice	LowTradePrice	CloseTradePrice	
dateTimeMaxHTP = [(0,0,0,0,0) for _ in range(len(dateWithOTP))] #[date,time,CallPut,SP,maxHTP]
prevDate = dateWithOTP[0][0]
N = len(df)
curDfLen = N
for dOTPidx, (date,_,CallPut,sp,_) in enumerate(dateWithOTP):
    #print("Checking for date={},CallPut={},sp={}".format(date,CallPut,sp))
    maxHTP = -1
    doSameDate = False
    for index,row in df.iloc[dfStartIdx:].iterrows():
        #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
        #print("htp={},maxHTP={},time={}".format(row[7],maxHTP,row[1]))
        if index == N-1: #index == N - dfStartIdx - 2:
            #needed this one here to check for end index, which can contain time inside the timeframe we want
            #print("if4.1-----------------------------------------")
            #case 1 - we're done with prev date
            dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

            #initialize variables for next iteration of dateWithOTP
            dfStartIdx = index + 1
            prevDate = row[0]
            maxHTP = -1
            break
            
        if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
            if index == N-1: #index == N - dfStartIdx - 2:
                #need this one here again because the last index may have time that is outside of timeframe
                #print("if4.1-----------------------------------------")
                #case 1 - we're done with prev date
                dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)
            
                #initialize variables for next iteration of dateWithOTP
                dfStartIdx = index + 1
                prevDate = row[0]
                maxHTP = -1
                break
            continue
        elif date == row[0] and CallPut == row[3] and sp == row[4]:
            #print("if1")
            #print("row: {}".format(list(row)))
            #Check if there's a new max HTP
            #print("htp={},maxHTP={},time={}".format(row[7],maxHTP,row[1]))
            if row[7] > maxHTP:
                #print("if2")
                #print("update for maxHTP: {}".format(row[7]))
                maxHTP = row[7]
                maxHTPTime = row[1]
        elif date != row[0]:
            #print("if3")
            #date doesn't match, so we're done searching for previous date OR this date has no value in df
            if maxHTP != -1: #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMaxHTP  
                #print("if4")
                #case 1 - we're done with prev date
                dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)
            
                #initialize variables for next iteration of dateWithOTP
                dfStartIdx = index + 1
                prevDate = row[0]
                maxHTP = -1
                break
            elif index == N-1: #index == N - dfStartIdx - 2:
                #print("if4.1-----------------------------------------")
                #case 1 - we're done with prev date
                dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)
            
                #initialize variables for next iteration of dateWithOTP
                dfStartIdx = index + 1
                prevDate = row[0]
                maxHTP = -1
                break
            else:
                #print('if5')
                #Case 2 - date has no value in df, so doesn't exist in dateWithOTP
                #keep iterating until we past this date and until we found the date that matches dateWithOTP
                dfStartIdx = index + 1
                #print("dfStartIdx: {}".format(dfStartIdx))
                for indexb,rowb in df.iloc[dfStartIdx:].iterrows():
                    if rowb[0] == date:
                        #print("if6")
                        prevDate = rowb[0]
                        dfStartIdx = indexb
                        doSameDate = True #found matching date so now iterate from that as startIdx to cover it instead of using current for loop
                        break
                if doSameDate:
                    #print("if7")
                    break
            

    if doSameDate:
        #print("if7.2")
        for index,row in df.iloc[dfStartIdx:].iterrows():
            #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
            if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                    #print("if11")
                    #reached the last index in df, so update for maxHTP
                    #case 1 - we're done with prev date
                    dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                    #initialize variables for next iteration of dateWithOTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    maxHTP = -1
                    break
                    
            if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                    #print("if11")
                    #reached the last index in df, so update for maxHTP
                    #case 1 - we're done with prev date
                    dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                    #initialize variables for next iteration of dateWithOTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    maxHTP = -1
                    break
                continue
            elif date == row[0] and CallPut == row[3] and sp == row[4]:
                #print("if8")
                #print("row: {}".format(list(row)))
                #Check if there's a new max HTP
                #print("htp={},maxHTP={},time={}".format(row[7],maxHTP,row[1]))
                if row[7] > maxHTP:
                    #print("if9")
                    #print("update for maxHTP: {}".format(row[7]))
                    maxHTP = row[7]
                    maxHTPTime = row[1]
            elif date != row[0]:
                #date doesn't match, so we're done searching for previous date OR this date has no value in df
                #print("if10")
                if maxHTP != -1: #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMaxHTP 
                    #case 1 - we're done with prev date
                    dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                    #initialize variables for next iteration of dateWithOTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    maxHTP = -1
                    break
            

In [26]:
dateTimeMaxHTP

[(20200304, '9:58:00 AM', 'P', 305, 3.52),
 (20200306, '10:51:00 AM', 'C', 294, 5.15),
 (20200311, '9:37:00 AM', 'C', 282, 3.5)]

#### Find the minimum of the option (LowTradePrice) from 9:37:00 AM to the maxHTP time
#### If the max time occurs at 9:37:00 AM then find the minimum LowTradePrice of the option from 9:37:00 AM to the end of the day

In [27]:
dfStartIdx = 0
#Date	TimeBarStart	Ticker	CallPut	Strike	ExpirationDate	OpenTradePrice	HighTradePrice	LowTradePrice	CloseTradePrice	
dateTimeMinLTP = [(0,0,0,0,0) for _ in range(len(dateTimeMaxHTP))] #[date,time,CallPut,SP,minLTP]
prevDate = dateTimeMaxHTP[0][0]
N = len(df)
curDfLen = N
for dTMidx, (date,maxHTPTime,CallPut,sp,_) in enumerate(dateTimeMaxHTP):
    
    while int(prevDate) < date:
        #if the row's date is not part of our dateTimeMaxHTP, then we keep moving to next row
        dfStartIdx += 1
        if dfStartIdx == N - 1:
            break
        prevDate = df.iloc[dfStartIdx][0]


    #print("Checking for date={},maxHTPTime={},CallPut={},sp={}".format(date,maxHTPTime,CallPut,sp))
    minLTP = float("inf")
    doSameDate = False
    
    if maxHTPTime == "9:37:00 AM":
        #print("inside maxHTPTime == 9:37:00 AM")
        #if the max time occurs at 9:37:00 AM then find the minimum LowTradePrice of the option from 9:37:00 AM to the end of the day
        for index,row in df.iloc[dfStartIdx:].iterrows():
            #print("index,dfStartIdx,N-dfStartIdx-1,time: {},{},{},{}".format(index,dfStartIdx,N-dfStartIdx-1,row[1]))
            #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
            if index == N-1: #index == N - dfStartIdx - 2:
                #case 1 - we're done with prev date
                #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                #initialize variables for next iteration of dateTimeMaxHTP
                dfStartIdx = index + 1
                prevDate = row[0]
                minLTP = float("inf")
                break

            #want 9:37AM to end of day
            #otherwise toss out this row
            if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                if index == N-1: #index == N - dfStartIdx - 2:
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break
                continue
            elif date == row[0] and CallPut == row[3] and sp == row[4]:
                #print("row: {}".format(list(row)))
                #Check if there's a new max HTP
                #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                if row[8] < minLTP:
                    #print("update for minLTP: {}".format(row[8]))
                    minLTP = row[8]
                    minLTPTime = row[1]
            elif date != row[0]:
                #date doesn't match, so we're done searching for previous date OR this date has no value in df
                if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break
                elif index == N-1: #index == N - dfStartIdx - 2:
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break
                else:
                    #print('if5 - date has no value in df')
                    #Case 2 - date has no value in df, so doesn't exist in dateTimeMaxHTP
                    #keep iterating until we past this date and until we found the date that matches dateTimeMaxHTP
                    dfStartIdx = index + 1
                    #print("dfStartIdx: {}".format(dfStartIdx))
                    for indexb,rowb in df.iloc[dfStartIdx:].iterrows():
                        if rowb[0] == date:
                            prevDate = rowb[0]
                            dfStartIdx = indexb
                            doSameDate = True #found matching date so now iterate from that as startIdx to cover it instead of using current for loop
                            break
                    if doSameDate:
                        break
            

            if doSameDate:
                for index,row in df.iloc[dfStartIdx:].iterrows():
                    #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
                    if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break

                    #want 9:37AM to end of day
                    #otherwise toss out this row
                    if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                        if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                            #reached the last index in df, so update for minLTP
                            #case 1 - we're done with prev date
                            #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                            dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                            #initialize variables for next iteration of dateTimeMaxHTP
                            dfStartIdx = index + 1
                            prevDate = row[0]
                            minLTP = float("inf")
                            break
                        continue
                    elif date == row[0] and CallPut == row[3] and sp == row[4]:
                        #print("row: {}".format(list(row)))
                        #Check if there's a new max HTP
                        #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                        if row[8] < minLTP:
                            #print("update for minLTP: {}".format(row[8]))
                            minLTP = row[8]
                            minLTPTime = row[1]
                    elif date != row[0]:
                        #date doesn't match, so we're done searching for previous date OR this date has no value in df
                        if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP 
                            #case 1 - we're done with prev date
                            #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                            dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                            #initialize variables for next iteration of dateTimeMaxHTP
                            dfStartIdx = index + 1
                            prevDate = row[0]
                            minLTP = float("inf")
                            break
    else:
        #print("NOT inside maxHTPTime == 9:37:00 AM")
        #max occurs after 9:37AM - find the minimum of the option (LowTradePrice) from 9:37:00 AM to the max HTP time
        for index,row in df.iloc[dfStartIdx:].iterrows():
            #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
            #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
            if index == N-1: #index == N - dfStartIdx - 2:
                #case 1 - we're done with prev date
                #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                #initialize variables for next iteration of dateTimeMaxHTP
                dfStartIdx = index + 1
                prevDate = row[0]
                minLTP = float("inf")
                break

            #want 9:37AM to maxHTP time
            #toss out others
            if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],maxHTPTime):
                if index == N-1: #index == N - dfStartIdx - 2:
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break
                continue
            elif date == row[0] and CallPut == row[3] and sp == row[4]:
                #print("row: {}".format(list(row)))
                #Check if there's a new max HTP
                #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                if row[8] < minLTP:
                    #print("update for minLTP: {}".format(row[8]))
                    minLTP = row[8]
                    minLTPTime = row[1]
            elif date != row[0]:
                #date doesn't match, so we're done searching for previous date OR this date has no value in df
                if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break
                elif index == N-1: #index == N - dfStartIdx - 2:
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break
                else:
                    #print('if5')
                    #Case 2 - date has no value in df, so doesn't exist in dateTimeMaxHTP
                    #keep iterating until we past this date and until we found the date that matches dateTimeMaxHTP
                    dfStartIdx = index + 1
                    #print("dfStartIdx: {}".format(dfStartIdx))
                    for indexb,rowb in df.iloc[dfStartIdx:].iterrows():
                        if rowb[0] == date:
                            prevDate = rowb[0]
                            dfStartIdx = indexb
                            doSameDate = True #found matching date so now iterate from that as startIdx to cover it instead of using current for loop
                            break
                    if doSameDate:
                        break
            

            if doSameDate:
                for index,row in df.iloc[dfStartIdx:].iterrows():
                    #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
                    if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        #print("prevDate={},minLTPTime={},CallPut={},sp={},minLTP={}".format(prevDate,minLTPTime,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break

                    if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],maxHTPTime):
                        if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                            #reached the last index in df, so update for minLTP
                            #case 1 - we're done with prev date
                            dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                            #initialize variables for next iteration of dateTimeMaxHTP
                            dfStartIdx = index + 1
                            prevDate = row[0]
                            minLTP = float("inf")
                            break
                        continue
                    elif date == row[0] and CallPut == row[3] and sp == row[4]:
                        #print("row: {}".format(list(row)))
                        #Check if there's a new max HTP
                        #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                        if row[8] < minLTP:
                            #print("update for minLTP: {}".format(row[8]))
                            minLTP = row[8]
                            minLTPTime = row[1]
                    elif date != row[0]:
                        #date doesn't match, so we're done searching for previous date OR this date has no value in df
                        if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP 
                            #case 1 - we're done with prev date
                            dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                            #initialize variables for next iteration of dateTimeMaxHTP
                            dfStartIdx = index + 1
                            prevDate = row[0]
                            minLTP = float("inf")
                            break

In [28]:
dateWithOTP

[(20200304, '9:36:00 AM', 'P', 305, 2.26),
 (20200306, '9:36:00 AM', 'C', 294, 2.87),
 (20200311, '9:36:00 AM', 'C', 282, 3.24)]

In [29]:
#[date,time,CallPut,sp,minLTP]
dateTimeMinLTP

[(20200304, '9:41:00 AM', 'P', 305, 2.22),
 (20200306, '9:53:00 AM', 'C', 294, 1.9),
 (20200311, '3:59:00 PM', 'C', 282, 0.01)]

In [30]:
dateTimeMaxHTP

[(20200304, '9:58:00 AM', 'P', 305, 3.52),
 (20200306, '10:51:00 AM', 'C', 294, 5.15),
 (20200311, '9:37:00 AM', 'C', 282, 3.5)]

In [31]:
numValidDates = len(dateWithOTP)
mergedOtpLtpHtp = [(0,"Z",0,("Z",0),("Z",0),("Z",0))] * numValidDates #(date,CallPut,SP,(time1,OTP),(time2,minLTP),(time3,maxHTP))
for index in range(numValidDates):
    date = dateWithOTP[index][0]
    CallPut = dateWithOTP[index][2]
    SP = dateWithOTP[index][3]
    time1, OTP = dateWithOTP[index][1], dateWithOTP[index][4]
    time2, minLTP = dateTimeMinLTP[index][1], dateTimeMinLTP[index][4]
    time3, maxHTP = dateTimeMaxHTP[index][1], dateTimeMaxHTP[index][4]
    mergedOtpLtpHtp[index] = (date,CallPut,SP,(time1,OTP),(time2,minLTP),(time3,maxHTP))
    
    

In [32]:
mergedOtpLtpHtp

[(20200304,
  'P',
  305,
  ('9:36:00 AM', 2.26),
  ('9:41:00 AM', 2.22),
  ('9:58:00 AM', 3.52)),
 (20200306,
  'C',
  294,
  ('9:36:00 AM', 2.87),
  ('9:53:00 AM', 1.9),
  ('10:51:00 AM', 5.15)),
 (20200311,
  'C',
  282,
  ('9:36:00 AM', 3.24),
  ('3:59:00 PM', 0.01),
  ('9:37:00 AM', 3.5))]

In [33]:
value = 1 - ((2.26 - 2.22) / 2.26 * 100)
formatted_string = "{:.3f}".format(value)
print(formatted_string + "%")

-0.770%


In [40]:
import collections
dropFromOTP = collections.defaultdict(int) #part 1
buyToMax = collections.defaultdict(int) #part 2
numValidDates = len(mergedOtpLtpHtp)
margin = 5 #this is our margin for our %
for index in range(numValidDates):
    time1, otp = mergedOtpLtpHtp[index][3][0], mergedOtpLtpHtp[index][3][1]
    time2, minLTP = mergedOtpLtpHtp[index][4][0], mergedOtpLtpHtp[index][4][1]
    time3, maxHTP = mergedOtpLtpHtp[index][5][0], mergedOtpLtpHtp[index][5][1]
    if afterTime(time2,time3):
        #failure date, we lose 50%
        dropPerc = -50
    else:
        #dropPerc = math.ceil((otp - minLTP) / otp * 100)
        dropPerc = (otp - minLTP) / otp * 100 # 93.222 --> 95
        dropPerc =  margin * math.ceil(dropPerc/margin)
    #print(otp,minLTP,dropPerc)
    dropFromOTP[dropPerc] += 1
    #print("otp={}, minLTP={}, dropPerc={}".format(otp,minLTP,dropPerc))
    
mostOccurredDropPerc = max(dropFromOTP, key=dropFromOTP.get)
print("mostOccurredDropPerc:{}".format(mostOccurredDropPerc))

for index in range(numValidDates):
    time1, otp = mergedOtpLtpHtp[index][3][0], mergedOtpLtpHtp[index][3][1]
    time2, minLTP = mergedOtpLtpHtp[index][4][0], mergedOtpLtpHtp[index][4][1]
    time3, maxHTP = mergedOtpLtpHtp[index][5][0], mergedOtpLtpHtp[index][5][1]
    #print("otp={}, minLTP={}, mostOccurredDropPerc={}, buyPriceGeneral={}".format(otp,minLTP,mostOccurredDropPerc,buyPriceGeneral))
    
    #percent profit from buyPriceGeneral --> maxHTP
    if afterTime(time2,time3):
        buyPriceGeneral = 0
        profitPerc = -50
    else:
        buyPriceGeneral = otp - mostOccurredDropPerc/100*otp
        profitPerc = math.floor((maxHTP - buyPriceGeneral) / maxHTP * 100)
    if buyPriceGeneral*0.5 >= minLTP:
        #if our minLTP can still drop very low (past 50% our buy) then we have a failure
        profitPerc = -50
    #print("otp={}, minLTP={}, mostOccurredDropPerc={}, buyPriceGeneral={}, profitPerc={}".format(otp,minLTP,mostOccurredDropPerc,buyPriceGeneral,profitPerc))
    buyToMax[profitPerc] += 1
temp = buyToMax[-50]
del buyToMax[-50]
mostOccuredBuyProfit = max(buyToMax, key=buyToMax.get) #excludes -50
print("mostOccuredBuyProfit:{}".format(mostOccuredBuyProfit))
buyToMax[-50] = temp

mostOccurredDropPerc:5
mostOccuredBuyProfit:39


In [42]:
print("dropFromOTP: {}".format(dropFromOTP))
print("buyToMax: {}".format(buyToMax))
print("mostOccurredDropPerc: {}%".format(mostOccurredDropPerc))
print("mostOccuredBuyProfit: {}%".format(mostOccuredBuyProfit))

dropFromOTP: defaultdict(<class 'int'>, {5: 1, 35: 1, -50: 1})
buyToMax: defaultdict(<class 'int'>, {39: 1, 47: 1, -50: 1})
mostOccurredDropPerc: 5%
mostOccuredBuyProfit: 39%


In [61]:
#takes in a dataset, and outputs date, color, SP, OTP, minLTP, and maxHTP
def datasetToMerged(dfValidation):
    #PART 1 GETS Color and StrikePrice
    dateTimeCandleSP = [] #(date,time,color,StrikePrice,StockPrice)
    date = dfValidation.iloc[0]["Date"]
    openPrice = dfValidation.iloc[0]["UnderOpenAskPrice"]
    skipDate = False
    #indices: 0=Date, 1=TimeBarStart, 13= UnderCloseAskPrice, 11=UnderOpenAskPrice
    for index,row in dfValidation.iterrows():
        #we are looking for the UnderOpenAskPrice(UOAP) at 9:30:00 AM (the first time it occurs) 
        #we are looking for the UnderOpenAskPrice (UOAP) at 9:36:00 AM (the last time it occurs)
        #then we subtract the 9:35:00 AM time by the 9:30:00 AM time

        if date == row[0]:
            if skipDate:
                continue
            #lastPrice = row["UnderCloseAskPrice"]
            prevRow = row
            #if time is past 9:35AM, then use the latest 9:35AM (if it doesn't exist, then use the current row's time)
            time = row[1]
            if not beforeTime(time,"9:36:00 AM"):
                #calculate starting price difference

                #if 9:35 exists
                if prevRow[1] == "9:35:00 AM":
                    #we take the UnderCloseAskPrice
                    lastPrice = prevRow[13]
                else:
                    #if 9:35 doesn't exist
                    #if 9:36 exists, then take the open price, otherwise skip this date
                    if time == "9:36:00 AM":
                        lastPrice = row[11]
                    else:
                        skipDate = True
                        continue
                diff = lastPrice - openPrice
                if math.isnan(diff):
                    #either openPrice or lastPrice doesn't exist
                    skipDate = True
                    continue
                #print("date={},lastPrice={}".format(date,lastPrice))
                if diff >= 0:
                    #green start
                    dateTimeCandleSP.append((date,time,"GREEN",math.ceil(lastPrice),lastPrice))
                else:
                    dateTimeCandleSP.append((date,time,"RED",math.floor(lastPrice),lastPrice))
                skipDate = True
        else:
            #we found new date, get the green/red bar for the previous date, reinitialize some variables
            date = row[0]
            openPrice = row[11]
            skipDate = False

    #PART 2 GETS OTP
    dtcspIdx = 0
    dfStartIdx = 0
    dateWithOTP = [] #(date, time, CallPut, SP, OTP)
    for dtcsp in dateTimeCandleSP:
        #print("dateWithOTP: {}".format(dateWithOTP))
        date, time, color, strikeP = dtcsp[0],dtcsp[1],dtcsp[2], dtcsp[3]
        for index,row in dfValidation.iloc[dfStartIdx:].iterrows():
            if color == "GREEN":
                #This is call, get OpenTradePrice(OTP) at 9:36AM
                #Match Date, Time, Call=C, StrikePrice
                if row[0] == date and row[1] == time and row[3] == "C" and row[4] == strikeP:
                    dateWithOTP.append((date,time,"C",strikeP,row[6]))
                    #update for next iteration of dtcsp
                    dfStartIdx = index + 1
                    break
            else:
                #This is put, get OpenTradePrice(OTP) at 9:36AM
                #Match Date, Time, Call=P, StrikePrice
                if row[0] == date and row[1] == time and row[3] == "P" and row[4] == strikeP:
                    dateWithOTP.append((date,time,"P",strikeP,row[6]))
                    #update for next iteration of dtcsp
                    dfStartIdx = index + 1
                    break
    #PART 4 GETS maxHTP
    dfStartIdx = 0
    #Date	TimeBarStart	Ticker	CallPut	Strike	ExpirationDate	OpenTradePrice	HighTradePrice	LowTradePrice	CloseTradePrice	
    dateTimeMaxHTP = [(0,0,0,0,0) for _ in range(len(dateWithOTP))] #[date,time,CallPut,SP,maxHTP]
    prevDate = dateWithOTP[0][0]
    N = len(dfValidation)
    curDfLen = N
    for dOTPidx, (date,_,CallPut,sp,_) in enumerate(dateWithOTP):
        #print("Checking for date={},CallPut={},sp={}".format(date,CallPut,sp))
        maxHTP = -1
        doSameDate = False
        for index,row in dfValidation.iloc[dfStartIdx:].iterrows():
            #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
            #print("htp={},maxHTP={},time={}".format(row[7],maxHTP,row[1]))
            if index == N-1: #index == N - dfStartIdx - 2:
                #needed this one here to check for end index, which can contain time inside the timeframe we want
                #print("if4.1-----------------------------------------")
                #case 1 - we're done with prev date
                dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                #initialize variables for next iteration of dateWithOTP
                dfStartIdx = index + 1
                prevDate = row[0]
                maxHTP = -1
                break

            if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                if index == N-1: #index == N - dfStartIdx - 2:
                    #need this one here again because the last index may have time that is outside of timeframe
                    #print("if4.1-----------------------------------------")
                    #case 1 - we're done with prev date
                    dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                    #initialize variables for next iteration of dateWithOTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    maxHTP = -1
                    break
                continue
            elif date == row[0] and CallPut == row[3] and sp == row[4]:
                #print("if1")
                #print("row: {}".format(list(row)))
                #Check if there's a new max HTP
                #print("htp={},maxHTP={},time={}".format(row[7],maxHTP,row[1]))
                if row[7] > maxHTP:
                    #print("if2")
                    #print("update for maxHTP: {}".format(row[7]))
                    maxHTP = row[7]
                    maxHTPTime = row[1]
            elif date != row[0]:
                #print("if3")
                #date doesn't match, so we're done searching for previous date OR this date has no value in dfValidation
                if maxHTP != -1: #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMaxHTP  
                    #print("if4")
                    #case 1 - we're done with prev date
                    dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                    #initialize variables for next iteration of dateWithOTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    maxHTP = -1
                    break
                elif index == N-1: #index == N - dfStartIdx - 2:
                    #print("if4.1-----------------------------------------")
                    #case 1 - we're done with prev date
                    dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                    #initialize variables for next iteration of dateWithOTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    maxHTP = -1
                    break
                else:
                    #print('if5')
                    #Case 2 - date has no value in dfValidation, so doesn't exist in dateWithOTP
                    #keep iterating until we past this date and until we found the date that matches dateWithOTP
                    dfStartIdx = index + 1
                    #print("dfStartIdx: {}".format(dfStartIdx))
                    for indexb,rowb in dfValidation.iloc[dfStartIdx:].iterrows():
                        if rowb[0] == date:
                            #print("if6")
                            prevDate = rowb[0]
                            dfStartIdx = indexb
                            doSameDate = True #found matching date so now iterate from that as startIdx to cover it instead of using current for loop
                            break
                    if doSameDate:
                        #print("if7")
                        break


        if doSameDate:
            #print("if7.2")
            for index,row in dfValidation.iloc[dfStartIdx:].iterrows():
                #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
                if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                        #print("if11")
                        #reached the last index in dfValidation, so update for maxHTP
                        #case 1 - we're done with prev date
                        dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                        #initialize variables for next iteration of dateWithOTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        maxHTP = -1
                        break

                if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                    if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                        #print("if11")
                        #reached the last index in dfValidation, so update for maxHTP
                        #case 1 - we're done with prev date
                        dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                        #initialize variables for next iteration of dateWithOTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        maxHTP = -1
                        break
                    continue
                elif date == row[0] and CallPut == row[3] and sp == row[4]:
                    #print("if8")
                    #print("row: {}".format(list(row)))
                    #Check if there's a new max HTP
                    #print("htp={},maxHTP={},time={}".format(row[7],maxHTP,row[1]))
                    if row[7] > maxHTP:
                        #print("if9")
                        #print("update for maxHTP: {}".format(row[7]))
                        maxHTP = row[7]
                        maxHTPTime = row[1]
                elif date != row[0]:
                    #date doesn't match, so we're done searching for previous date OR this date has no value in dfValidation
                    #print("if10")
                    if maxHTP != -1: #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMaxHTP 
                        #case 1 - we're done with prev date
                        dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                        #initialize variables for next iteration of dateWithOTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        maxHTP = -1
                        break
    #PART 3 GETS minLTP
    dfStartIdx = 0
    #Date	TimeBarStart	Ticker	CallPut	Strike	ExpirationDate	OpenTradePrice	HighTradePrice	LowTradePrice	CloseTradePrice	
    dateTimeMinLTP = [(0,0,0,0,0) for _ in range(len(dateTimeMaxHTP))] #[date,time,CallPut,SP,minLTP]
    prevDate = dateTimeMaxHTP[0][0]
    N = len(dfValidation)
    curDfLen = N
    for dTMidx, (date,maxHTPTime,CallPut,sp,_) in enumerate(dateTimeMaxHTP):

        while int(prevDate) < date:
            #if the row's date is not part of our dateTimeMaxHTP, then we keep moving to next row
            dfStartIdx += 1
            if dfStartIdx == N - 1:
                break
            prevDate = dfValidation.iloc[dfStartIdx][0]


        #print("Checking for date={},maxHTPTime={},CallPut={},sp={}".format(date,maxHTPTime,CallPut,sp))
        minLTP = float("inf")
        doSameDate = False

        if maxHTPTime == "9:37:00 AM":
            #print("inside maxHTPTime == 9:37:00 AM")
            #if the max time occurs at 9:37:00 AM then find the minimum LowTradePrice of the option from 9:37:00 AM to the end of the day
            for index,row in dfValidation.iloc[dfStartIdx:].iterrows():
                #print("index,dfStartIdx,N-dfStartIdx-1,time: {},{},{},{}".format(index,dfStartIdx,N-dfStartIdx-1,row[1]))
                #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                if index == N-1: #index == N - dfStartIdx - 2:
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break

                #want 9:37AM to end of day
                #otherwise toss out this row
                if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                    if index == N-1: #index == N - dfStartIdx - 2:
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break
                    continue
                elif date == row[0] and CallPut == row[3] and sp == row[4]:
                    #print("row: {}".format(list(row)))
                    #Check if there's a new max HTP
                    #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                    if row[8] < minLTP:
                        #print("update for minLTP: {}".format(row[8]))
                        minLTP = row[8]
                        minLTPTime = row[1]
                elif date != row[0]:
                    #date doesn't match, so we're done searching for previous date OR this date has no value in dfValidation
                    if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break
                    elif index == N-1: #index == N - dfStartIdx - 2:
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break
                    else:
                        #print('if5 - date has no value in dfValidation')
                        #Case 2 - date has no value in dfValidation, so doesn't exist in dateTimeMaxHTP
                        #keep iterating until we past this date and until we found the date that matches dateTimeMaxHTP
                        dfStartIdx = index + 1
                        #print("dfStartIdx: {}".format(dfStartIdx))
                        for indexb,rowb in dfValidation.iloc[dfStartIdx:].iterrows():
                            if rowb[0] == date:
                                prevDate = rowb[0]
                                dfStartIdx = indexb
                                doSameDate = True #found matching date so now iterate from that as startIdx to cover it instead of using current for loop
                                break
                        if doSameDate:
                            break


                if doSameDate:
                    for index,row in dfValidation.iloc[dfStartIdx:].iterrows():
                        #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
                        if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                            #case 1 - we're done with prev date
                            #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                            dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                            #initialize variables for next iteration of dateTimeMaxHTP
                            dfStartIdx = index + 1
                            prevDate = row[0]
                            minLTP = float("inf")
                            break

                        #want 9:37AM to end of day
                        #otherwise toss out this row
                        if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                            if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                                #reached the last index in dfValidation, so update for minLTP
                                #case 1 - we're done with prev date
                                #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                                dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                                #initialize variables for next iteration of dateTimeMaxHTP
                                dfStartIdx = index + 1
                                prevDate = row[0]
                                minLTP = float("inf")
                                break
                            continue
                        elif date == row[0] and CallPut == row[3] and sp == row[4]:
                            #print("row: {}".format(list(row)))
                            #Check if there's a new max HTP
                            #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                            if row[8] < minLTP:
                                #print("update for minLTP: {}".format(row[8]))
                                minLTP = row[8]
                                minLTPTime = row[1]
                        elif date != row[0]:
                            #date doesn't match, so we're done searching for previous date OR this date has no value in dfValidation
                            if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP 
                                #case 1 - we're done with prev date
                                #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                                dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                                #initialize variables for next iteration of dateTimeMaxHTP
                                dfStartIdx = index + 1
                                prevDate = row[0]
                                minLTP = float("inf")
                                break
        else:
            #print("NOT inside maxHTPTime == 9:37:00 AM")
            #max occurs after 9:37AM - find the minimum of the option (LowTradePrice) from 9:37:00 AM to the max HTP time
            for index,row in dfValidation.iloc[dfStartIdx:].iterrows():
                #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
                #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                if index == N-1: #index == N - dfStartIdx - 2:
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break

                #want 9:37AM to maxHTP time
                #toss out others
                if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],maxHTPTime):
                    if index == N-1: #index == N - dfStartIdx - 2:
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break
                    continue
                elif date == row[0] and CallPut == row[3] and sp == row[4]:
                    #print("row: {}".format(list(row)))
                    #Check if there's a new max HTP
                    #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                    if row[8] < minLTP:
                        #print("update for minLTP: {}".format(row[8]))
                        minLTP = row[8]
                        minLTPTime = row[1]
                elif date != row[0]:
                    #date doesn't match, so we're done searching for previous date OR this date has no value in dfValidation
                    if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break
                    elif index == N-1: #index == N - dfStartIdx - 2:
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break
                    else:
                        #print('if5')
                        #Case 2 - date has no value in dfValidation, so doesn't exist in dateTimeMaxHTP
                        #keep iterating until we past this date and until we found the date that matches dateTimeMaxHTP
                        dfStartIdx = index + 1
                        #print("dfStartIdx: {}".format(dfStartIdx))
                        for indexb,rowb in dfValidation.iloc[dfStartIdx:].iterrows():
                            if rowb[0] == date:
                                prevDate = rowb[0]
                                dfStartIdx = indexb
                                doSameDate = True #found matching date so now iterate from that as startIdx to cover it instead of using current for loop
                                break
                        if doSameDate:
                            break


                if doSameDate:
                    for index,row in dfValidation.iloc[dfStartIdx:].iterrows():
                        #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
                        if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                            #case 1 - we're done with prev date
                            #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                            #print("prevDate={},minLTPTime={},CallPut={},sp={},minLTP={}".format(prevDate,minLTPTime,CallPut,sp,minLTP))
                            dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                            #initialize variables for next iteration of dateTimeMaxHTP
                            dfStartIdx = index + 1
                            prevDate = row[0]
                            minLTP = float("inf")
                            break

                        if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],maxHTPTime):
                            if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                                #reached the last index in dfValidation, so update for minLTP
                                #case 1 - we're done with prev date
                                dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                                #initialize variables for next iteration of dateTimeMaxHTP
                                dfStartIdx = index + 1
                                prevDate = row[0]
                                minLTP = float("inf")
                                break
                            continue
                        elif date == row[0] and CallPut == row[3] and sp == row[4]:
                            #print("row: {}".format(list(row)))
                            #Check if there's a new max HTP
                            #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                            if row[8] < minLTP:
                                #print("update for minLTP: {}".format(row[8]))
                                minLTP = row[8]
                                minLTPTime = row[1]
                        elif date != row[0]:
                            #date doesn't match, so we're done searching for previous date OR this date has no value in dfValidation
                            if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP 
                                #case 1 - we're done with prev date
                                dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                                #initialize variables for next iteration of dateTimeMaxHTP
                                dfStartIdx = index + 1
                                prevDate = row[0]
                                minLTP = float("inf")
                                break
    
    #MERGE ALL 4 PARTS
    #part 1 - dateTimeCandleSP = [] #(date,time,color,StrikePrice,StockPrice)
    #part 2 - dateWithOTP = [] #(date, time, CallPut, SP, OTP)
    #part 3 - dateTimeMinLTP = [(0,0,0,0,0) for _ in range(len(dateTimeMaxHTP))] #[date,time,CallPut,SP,minLTP]
    #part 4 - dateTimeMaxHTP = [(0,0,0,0,0) for _ in range(len(dateWithOTP))] #[date,time,CallPut,SP,maxHTP]
    numDates = len(dateTimeCandleSP)
    result = [("Z","Z",-1,-1,-1) for _ in range(numDates)] #(date,color,SP,OTP,minLTP,maxHTP)
    for index in range(numDates):
        date = dateTimeCandleSP[index][0]
        color = dateTimeCandleSP[index][2]
        SP = dateWithOTP[index][3]
        OTP = dateWithOTP[index][4]
        minLTP = dateTimeMinLTP[index][4]
        maxHTP = dateTimeMaxHTP[index][4]
        result[index] = (date,color,SP,OTP,minLTP,maxHTP)
    return result

In [62]:
dfValidation = pd.read_csv('Jan 2020 - Dec 2020 merged SPY.csv')
result = datasetToMerged(dfValidation)
result #(date,color,SP,OTP,minLTP,maxHTP)

[(20200103, 'GREEN', 323, 0.43, 0.2, 0.7),
 (20200106, 'GREEN', 322, 0.31, 0.24, 1.65),
 (20200108, 'GREEN', 324, 0.39, 0.21, 1.78),
 (20200110, 'RED', 326, 0.21, 0.07, 0.81),
 (20200113, 'RED', 326, 0.44, 0.34, 0.55),
 (20200115, 'GREEN', 329, 0.19, 0.16, 0.44),
 (20200117, 'RED', 331, 0.3, 0.33, 0.41),
 (20200121, 'GREEN', 332, 0.23, 0.07, 0.36),
 (20200122, 'GREEN', 333, 0.23, 0.22, 0.32),
 (20200124, 'RED', 332, 0.36, 0.34, 4.6),
 (20200127, 'GREEN', 324, 1.06, 1.0, 1.49),
 (20200129, 'RED', 328, 0.69, 0.69, 1.9),
 (20200131, 'RED', 326, 0.87, 0.6, 5.15),
 (20200203, 'GREEN', 325, 0.5, 0.51, 1.5),
 (20200205, 'RED', 331, 0.43, 0.26, 0.85),
 (20200207, 'RED', 332, 0.71, 0.42, 0.81),
 (20200210, 'GREEN', 333, 0.31, 0.26, 1.64),
 (20200212, 'GREEN', 338, 0.14, 0.14, 0.2),
 (20200214, 'GREEN', 338, 0.37, 0.25, 0.32),
 (20200218, 'GREEN', 337, 0.65, 0.43, 0.65),
 (20200219, 'GREEN', 338, 0.45, 0.42, 1.1),
 (20200221, 'RED', 335, 0.98, 0.71, 2.41),
 (20200224, 'GREEN', 325, 1.28, 1.32, 1

In [107]:
profits = [] #(date,profit)
printCount = 0 #for debug
#these are computed previously from a training dataset
#mostOccurredDropPerc:5
#mostOccuredBuyProfit:39

netProfit = 1
dfStartIdx = 0
#indices: 0=Date, 1=TimeBarStart, 13= UnderCloseAskPrice, 11=UnderOpenAskPrice
for date,color,SP,OTP,minLTP,maxHTP in result:
    print("Finding profit for date={}".format(date))
    #keep iterating row startIdx forward until we find matching date
    for index,row in dfValidation.iloc[dfStartIdx:].iterrows():
        if row[0] != date:
            continue
        else:
            dfStartIdx = index
            break
    
    if color == "GREEN":
        callType = "C"
    else:
        callType = "P"
    
    #if our calculated price is > LTP then we successfully bought the option
    buyPrice = OTP * (1-mostOccurredDropPerc/100)
    bought = False
    for index,row in dfValidation.iloc[dfStartIdx:].iterrows():
        #printCount += 1
        #if printCount % 10**4:
        #    print("date={},time={}".format(row[0],row[1]))
        #print("date={},time={}".format(row[0],row[1]))
        soldThisMinute = False
        if callType == row[3] and SP == row[4] and afterTime(row[1],"9:36:00 AM") and beforeTime(row[1],"4:00:00 PM"):
            if row[1] == "3:59:00 PM":
                #This is our last time of the date that we're considering, so if we boughtsomething, then sell it
                if bought:
                    #sell what we bought at OTP from 3:59:00 PM
                    profit = row[6]/buyPrice #currentTimeOTP/buyPrice
                    netProfit *= profit
                    profits.append((date,profit))
                    print("Sold at: date={}, profit={}, buyPrice={}, sellPrice={}".format(date,profit,buyPrice,row[6]))
                    #buyPrice --> OTP
                    #5 --> 11 #2.2
                    #11 --> 5 #0.45
                    #[1.3,0.9,1.2] 1.3*0.9*1.2*(0.5)^failcount*(mostOccuredBuyProfit)^successCount
                    #1.3,1.5,0.9,0.5^2,1.39^5
                    #1.3*1.5
                    soldThisMinute = True
                    
                #reinitialize variables for next date
                dfStartIdx = index
                print("1. done with date={}".format(row[0]))
                break
                
            #if this date doesn't have 3:59PM with the desired constraints, then we will pretend that we didn't buy previously
            elif afterTime(row[1],"3:59:00 PM"):
                dfStartIdx = index
                print("2. done with date={}".format(row[0]))
                break
            elif bought:
                #see if this is sellable (i.e. sellGoal < HTP)
                if sellGoal < row[7]:
                    #successfully sold at this price, so obtain the profit
                    #our profit % is mostOccuredBuyProfit, so tally the success
                    #successCount += 1
                    profit = 1+mostOccuredBuyProfit/100
                    netProfit *= profit
                    profits.append((date,profit))
                    print("Sold at profit: date={}, profit={}, boughtAt={}, soldAt={}, HTP={}".format(date,profit,buyPrice,sellGoal,row[7]))
                    bought = False
                    soldThisMinute = True
                elif buyPrice * 0.5 >= row[8]:
                    #we hit -50% limit, so sell it for that loss before we lose more
                    netProfit *= 0.5
                    profit = 0.5
                    profits.append((date,profit))
                    print("Sold at loss: date={}, profit={}, buyPrice={}, sellPrice={}, LTP={}".format(date,profit,buyPrice,buyPrice/2,row[8]))
                    bought = False
                    soldThisMinute = True
                #otherwise keep holding
            
            if not bought and not soldThisMinute:
                if buyPrice > row[8]: #buyprice > LTP then we successfully bought the option
                    print("buyPrice={},LTP={}".format(buyPrice,row[8]))
                    bought = True
                    print("bought at date={},time={}".format(date,row[1]))
                    #We now look to sell it for buyPrice*(1+mostOccuredBuyProfit)
                    sellGoal = buyPrice*(1+mostOccuredBuyProfit/100)
        #if we past the desired time, then end the day
        if afterTime(row[1],"3:59:00 PM"):
            #reinitialize variables for next date
            dfStartIdx = index
            print("1. done with date={}".format(row[0]))
            break
        

Finding profit for date=20200103
buyPrice=0.4085,LTP=0.36
bought at date=20200103,time=9:37:00 AM
Sold at profit: date=20200103, profit=1.3900000000000001, boughtAt=0.4085, soldAt=0.5678150000000001, HTP=0.62
buyPrice=0.4085,LTP=0.4
bought at date=20200103,time=9:55:00 AM
Sold at profit: date=20200103, profit=1.3900000000000001, boughtAt=0.4085, soldAt=0.5678150000000001, HTP=0.58
buyPrice=0.4085,LTP=0.31
bought at date=20200103,time=10:00:00 AM
Sold at profit: date=20200103, profit=1.3900000000000001, boughtAt=0.4085, soldAt=0.5678150000000001, HTP=0.58
buyPrice=0.4085,LTP=0.32
bought at date=20200103,time=10:50:00 AM
Sold at loss: date=20200103, profit=0.5, buyPrice=0.4085, sellPrice=0.20425, LTP=0.2
buyPrice=0.4085,LTP=0.21
bought at date=20200103,time=11:11:00 AM
1. done with date=20200103
Finding profit for date=20200106
buyPrice=0.2945,LTP=0.24
bought at date=20200106,time=9:38:00 AM
Sold at profit: date=20200106, profit=1.3900000000000001, boughtAt=0.2945, soldAt=0.409355, HTP=0

Sold at loss: date=20200113, profit=0.5, buyPrice=0.418, sellPrice=0.209, LTP=0.07
buyPrice=0.418,LTP=0.07
bought at date=20200113,time=11:55:00 AM
Sold at loss: date=20200113, profit=0.5, buyPrice=0.418, sellPrice=0.209, LTP=0.06
buyPrice=0.418,LTP=0.06
bought at date=20200113,time=11:57:00 AM
Sold at loss: date=20200113, profit=0.5, buyPrice=0.418, sellPrice=0.209, LTP=0.06
buyPrice=0.418,LTP=0.04
bought at date=20200113,time=11:59:00 AM
1. done with date=20200113
Finding profit for date=20200115
buyPrice=0.1805,LTP=0.17
bought at date=20200115,time=9:37:00 AM
Sold at profit: date=20200115, profit=1.3900000000000001, boughtAt=0.1805, soldAt=0.25089500000000003, HTP=0.27
1. done with date=20200115
Finding profit for date=20200117
buyPrice=0.285,LTP=0.27
bought at date=20200117,time=9:49:00 AM
1. done with date=20200117
Finding profit for date=20200121
buyPrice=0.2185,LTP=0.19
bought at date=20200121,time=9:37:00 AM
Sold at loss: date=20200121, profit=0.5, buyPrice=0.2185, sellPrice=0.

buyPrice=0.8265,LTP=0.77
bought at date=20200131,time=9:37:00 AM
Sold at profit: date=20200131, profit=1.3900000000000001, boughtAt=0.8265, soldAt=1.148835, HTP=1.16
buyPrice=0.8265,LTP=0.75
bought at date=20200131,time=9:58:00 AM
Sold at profit: date=20200131, profit=1.3900000000000001, boughtAt=0.8265, soldAt=1.148835, HTP=1.16
1. done with date=20200131
Finding profit for date=20200203
buyPrice=0.475,LTP=0.47
bought at date=20200203,time=11:32:00 AM
Sold at loss: date=20200203, profit=0.5, buyPrice=0.475, sellPrice=0.2375, LTP=0.23
buyPrice=0.475,LTP=0.21
bought at date=20200203,time=11:51:00 AM
Sold at loss: date=20200203, profit=0.5, buyPrice=0.475, sellPrice=0.2375, LTP=0.2
buyPrice=0.475,LTP=0.23
bought at date=20200203,time=11:53:00 AM
1. done with date=20200203
Finding profit for date=20200205
buyPrice=0.4085,LTP=0.36
bought at date=20200205,time=9:48:00 AM
Sold at profit: date=20200205, profit=1.3900000000000001, boughtAt=0.4085, soldAt=0.5678150000000001, HTP=0.71
buyPrice=0

Sold at loss: date=20200207, profit=0.5, buyPrice=0.6745, sellPrice=0.33725, LTP=0.15
buyPrice=0.6745,LTP=0.15
bought at date=20200207,time=11:58:00 AM
Sold at loss: date=20200207, profit=0.5, buyPrice=0.6745, sellPrice=0.33725, LTP=0.15
1. done with date=20200207
Finding profit for date=20200210
buyPrice=0.2945,LTP=0.29
bought at date=20200210,time=9:48:00 AM
Sold at profit: date=20200210, profit=1.3900000000000001, boughtAt=0.2945, soldAt=0.409355, HTP=0.45
1. done with date=20200210
Finding profit for date=20200212
buyPrice=0.133,LTP=0.13
bought at date=20200212,time=9:43:00 AM
Sold at loss: date=20200212, profit=0.5, buyPrice=0.133, sellPrice=0.0665, LTP=0.06
buyPrice=0.133,LTP=0.06
bought at date=20200212,time=10:09:00 AM
Sold at loss: date=20200212, profit=0.5, buyPrice=0.133, sellPrice=0.0665, LTP=0.06
buyPrice=0.133,LTP=0.06
bought at date=20200212,time=10:11:00 AM
Sold at loss: date=20200212, profit=0.5, buyPrice=0.133, sellPrice=0.0665, LTP=0.06
buyPrice=0.133,LTP=0.07
bought

buyPrice=0.6174999999999999,LTP=0.43
bought at date=20200218,time=9:37:00 AM
Sold at loss: date=20200218, profit=0.5, buyPrice=0.6174999999999999, sellPrice=0.30874999999999997, LTP=0.3
buyPrice=0.6174999999999999,LTP=0.17
bought at date=20200218,time=10:29:00 AM
Sold at loss: date=20200218, profit=0.5, buyPrice=0.6174999999999999, sellPrice=0.30874999999999997, LTP=0.18
buyPrice=0.6174999999999999,LTP=0.24
bought at date=20200218,time=10:31:00 AM
Sold at loss: date=20200218, profit=0.5, buyPrice=0.6174999999999999, sellPrice=0.30874999999999997, LTP=0.21
buyPrice=0.6174999999999999,LTP=0.21
bought at date=20200218,time=10:33:00 AM
Sold at loss: date=20200218, profit=0.5, buyPrice=0.6174999999999999, sellPrice=0.30874999999999997, LTP=0.23
buyPrice=0.6174999999999999,LTP=0.2
bought at date=20200218,time=10:35:00 AM
Sold at loss: date=20200218, profit=0.5, buyPrice=0.6174999999999999, sellPrice=0.30874999999999997, LTP=0.19
buyPrice=0.6174999999999999,LTP=0.19
bought at date=20200218,ti

buyPrice=0.4275,LTP=0.42
bought at date=20200219,time=10:13:00 AM
Sold at profit: date=20200219, profit=1.3900000000000001, boughtAt=0.4275, soldAt=0.594225, HTP=0.62
1. done with date=20200219
Finding profit for date=20200221
buyPrice=0.9309999999999999,LTP=0.81
bought at date=20200221,time=9:37:00 AM
Sold at profit: date=20200221, profit=1.3900000000000001, boughtAt=0.9309999999999999, soldAt=1.29409, HTP=1.37
buyPrice=0.9309999999999999,LTP=0.91
bought at date=20200221,time=11:30:00 AM
1. done with date=20200221
Finding profit for date=20200224
buyPrice=1.216,LTP=1.2
bought at date=20200224,time=10:20:00 AM
Sold at loss: date=20200224, profit=0.5, buyPrice=1.216, sellPrice=0.608, LTP=0.57
buyPrice=1.216,LTP=0.59
bought at date=20200224,time=11:05:00 AM
Sold at loss: date=20200224, profit=0.5, buyPrice=1.216, sellPrice=0.608, LTP=0.59
buyPrice=1.216,LTP=0.62
bought at date=20200224,time=11:07:00 AM
Sold at loss: date=20200224, profit=0.5, buyPrice=1.216, sellPrice=0.608, LTP=0.6
buyP

buyPrice=2.147,LTP=2.09
bought at date=20200304,time=10:13:00 AM
1. done with date=20200304
Finding profit for date=20200306
buyPrice=2.7265,LTP=2.61
bought at date=20200306,time=9:41:00 AM
Sold at profit: date=20200306, profit=1.3900000000000001, boughtAt=2.7265, soldAt=3.7898350000000005, HTP=3.96
buyPrice=2.7265,LTP=2.69
bought at date=20200306,time=11:44:00 AM
1. done with date=20200306
Finding profit for date=20200311
buyPrice=3.078,LTP=2.96
bought at date=20200311,time=9:38:00 AM
Sold at loss: date=20200311, profit=0.5, buyPrice=3.078, sellPrice=1.539, LTP=1.5
buyPrice=3.078,LTP=1.64
bought at date=20200311,time=10:38:00 AM
Sold at loss: date=20200311, profit=0.5, buyPrice=3.078, sellPrice=1.539, LTP=1.5
buyPrice=3.078,LTP=1.33
bought at date=20200311,time=10:43:00 AM
Sold at loss: date=20200311, profit=0.5, buyPrice=3.078, sellPrice=1.539, LTP=1.38
buyPrice=3.078,LTP=1.44
bought at date=20200311,time=10:45:00 AM
Sold at loss: date=20200311, profit=0.5, buyPrice=3.078, sellPrice=

In [108]:
profits

[(20200103, 1.3900000000000001),
 (20200103, 1.3900000000000001),
 (20200103, 1.3900000000000001),
 (20200103, 0.5),
 (20200106, 1.3900000000000001),
 (20200106, 1.3900000000000001),
 (20200108, 1.3900000000000001),
 (20200108, 1.3900000000000001),
 (20200108, 1.3900000000000001),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200110, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5),
 (20200113, 0.5)

In [91]:
mostOccurredDropPerc
mostOccuredBuyProfit

39

In [109]:
netProfit

1.2352705272145751e-113